In [0]:
temp_sale_df = spark.sql("""
                         select 
  cast(OrderId as int) as OrderId,
  product,
  cast(QuantityOrdered as int) as QuantityOrdered,
  cast(PriceEach as int) as PriceEach,
  to_timestamp(OrderDate, 'MM/dd/yy H:mm') as OrderDate,
  PurchaseAddress,
  substr(split(PurchaseAddress, ',')[1],2) as City,
  substr(split(PurchaseAddress, ',')[2],2,2) as State,
  year(to_timestamp(OrderDate, 'MM/dd/yy H:mm')) as ReportYear,
  month(to_timestamp(OrderDate, 'MM/dd/yy H:mm')) as Month
from sales
where orderID != "Order ID" and orderid != "null"
                    """)

In [0]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS tmp_sale (
        OrderId INT,
        Product STRING,
        QuantityOrdered INT,
        PriceEach INT,
        OrderDate TIMESTAMP,
        PurchaseAddress STRING,
        City STRING,
        State STRING,
        ReportYear INT,
        Month INT
    )
    USING PARQUET
    LOCATION 'dbfs:/FileStore/salesdata4/tmp_sale'
""")

In [0]:
temp_sale_df.createOrReplaceTempView("temp_sale_view")

In [0]:
spark.sql("""
    INSERT INTO tmp_sale
    SELECT * FROM temp_sale_view
""")

In [0]:
spark.sql("""
    CREATE  TABLE sales3 (
        OrderId INT,
        Product STRING,
        Quantity INT,
        PriceEach INT,
        OrderDate TIMESTAMP,
        StoreAddress STRING,
        City STRING,
        State STRING,
        ReportYear INT,
        months INT
    )
    USING PARQUET
    PARTITIONED BY (ReportYear, months)
    OPTIONS ('compression' = 'snappy')
    LOCATION 'dbfs:/FileStore/salesdata4/published'
""")

In [0]:
spark.sql("""INSERT INTO sales3
select OrderId,
product,
QuantityOrdered,
PriceEach,
OrderDate,
PurchaseAddress,
City,
State,
ReportYear,
Month
From tmp_sale
""")

In [0]:
%sql
select * from sales3;

In [0]:
%sql
select * from sales;

In [0]:
%sql
use sales_db;
show tables;

In [0]:
spark.sql("""
    SELECT COUNT(*) AS total_orders_in_newyork
    FROM sales3
    WHERE LOWER(City) = 'new york city'
""").show()

In [0]:
spark.sql("""
    SELECT City, COUNT(*) AS total_orders
    FROM sales3
    GROUP BY City
    ORDER BY total_orders DESC
""").show(truncate=False)

# Q1. Which is the best month of the sales done ? [sales = price * quantity]


In [0]:
%sql

select months, round(sum((priceeach * quantity)),2) as totalsales
from sales3
group by months 
order by months asc;

# Q2. Which city sold the most products

In [0]:
%sql

select city,sum(quantity) as totalproducts
from sales3
group by City
order by totalproducts desc;